# Scikit-learn MNIST Model Deployment

 * Wrap a Scikit-learn MNIST python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

## Train locally
 

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, metrics
from sklearn.utils import shuffle
from sklearn.datasets import fetch_mldata
from sklearn.externals import joblib

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

mnist_images = mnist.train.images
mnist_labels = mnist.train.labels
 # To apply a classifier on this data, we need to flatten the image, to
    # turn the data in a (samples, feature) matrix:
n_samples = len(mnist_images)
data = mnist_images.reshape((n_samples, -1))
targets = mnist_labels

data,targets = shuffle(data,targets)
classifier = RandomForestClassifier(n_estimators=30)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples // 2], targets[:n_samples // 2])

# Now predict the value of the digit on the second half:
expected = targets[n_samples // 2:]
test_data = data[n_samples // 2:]

print(classifier.score(test_data, expected))

predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

joblib.dump(classifier, 'sk.pkl')



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0.9552363636363637
Classification report for classifier RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n

['sk.pkl']

Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python3:0.13 sk-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [3]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 sk-mnist:0.1

a8adfe1e38ed793348c8889a3eb7db82602d77da14b4a9c81cb69105cc32fbf8


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.312 0.488 0.971 0.609 0.807 0.484 0.306 0.549 0.293 0.224 0.394 0.242
  0.447 0.501 0.642 0.975 0.627 0.775 0.575 0.062 0.206 0.249 0.848 0.126
  0.022 0.435 0.441 0.292 0.968 0.456 0.288 0.703 0.026 0.064 0.159 0.782
  0.098 0.33  0.8   0.023 0.642 0.678 0.637 0.151 0.588 0.257 0.812 0.262
  0.9   0.978 0.531 0.769 0.042 0.595 0.042 0.335 0.521 0.427 0.246 0.167
  0.602 0.081 0.845 0.664 0.98  0.215 0.331 0.439 0.754 0.649 0.525 0.685
  0.964 0.771 0.16  0.787 0.23  0.533 0.349 0.909 0.016 0.16  0.522 0.632
  0.804 0.091 0.871 0.36  0.563 0.027 0.154 0.963 0.221 0.224 0.836 0.504
  0.547 0.624 0.192 0.456 0.238 0.129 0.755 0.132 0.638 0.191 0.393 0.377
  0.985 0.425 0.634 0.577 0.567 0.446 0.083 0.673 0.757 0.834 0.93  0.32
  0.435 0.34  0.927 0.91  0.533 0.937 0.458 0.463 0.077 0.652 0.516 0.113
  0.946 0.952 0.314 0.744 0.125 0.751 0.92  0.322 0.404 0.279 0.258 0.979
  0.268 0.443 0.773 0.032 0.397 0.871 0.554 0.664 

In [5]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [8]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Follow the instructions to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core).

## Wrap Model and Test

In [10]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.13 sk-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [11]:
!kubectl create -f sk_mnist.json

seldondeployment.machinelearning.seldon.io/sk-mnist created


In [12]:
!kubectl rollout status deploy/sk-mnist-single-model-3812de6

Waiting for deployment "sk-mnist-single-model-3812de6" rollout to finish: 0 of 1 updated replicas are available...
deployment "sk-mnist-single-model-3812de6" successfully rolled out


In [13]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    sk-mnist --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[0.445 0.858 0.358 0.545 0.91  0.803 0.061 0.822 0.634 0.225 0.17  0.938
  0.754 0.539 0.075 0.548 0.34  0.083 0.858 0.835 0.755 0.041 0.058 0.414
  0.631 0.897 0.989 0.486 0.62  0.366 0.596 0.74  0.667 0.347 0.923 0.992
  0.54  0.88  0.521 0.877 0.945 0.727 0.697 0.543 0.714 0.147 0.361 0.21
  1.    0.742 0.666 0.86  0.59  0.607 0.146 0.345 0.342 0.751 0.958 0.869
  0.122 0.779 0.468 0.29  0.79  0.857 0.501 0.032 0.466 0.678 0.823 0.268
  0.598 0.27  0.55  0.648 0.818 0.732 0.344 0.123 0.735 0.741 0.675 0.68
  0.777 0.584 0.104 0.72  0.237 0.237 0.465 0.625 0.705 0.09  0.673 0.422
  0.51  0.603 0.271 0.957 0.565 1.    0.441 0.05  0.772 0.4   0.306 0.186
  0.822 0.103 0.432 0.151 0.232 0.134 0.163 0.826 0.01  0.756 0.898 0.918
  0.783 0.185 0.696 0.194 0.595 0.668 0.869 0.165 0.541 0.287 0.52  0.215
  0.007 0.564 0.643 0.809 0.307 0.973 0.671 0.552 0.073 0.843 0.472 0.514
  0.2   0.606 0.387 0.23  0.749 0.161 0.898 0.109 0

In [17]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
